# Többváltozós analízis mérnöki alkalmazásai - 7. gyakorlat
## Lineáris egyenletrendszerek, végeselem módszer

Az óra elején használt VEM-es anyagokat ezen a [linken](https://drive.google.com/open?id=11QzHA1ODJIydrWlst7KUO-jkWL9gxBo1) találjátok meg. A Cramer-szabály szemléltetését ebben a [videóban](https://www.youtube.com/watch?v=jBsC34PxzoM&t=550s) nézhetitek meg megint.

Láttuk, hogy LER-eket gyorsan megoldani nagyon fontos. Nézzük meg, hogy hogyan megy ez Pythonban, és alkalmazzuk a frissen megszerzett tudásunkat! A végeselem módszer alapjait ebben a kódban nem részletezem, a VEM-es anyagokban minden le van írva - jobban is, mint ahogy én tudom.


In [2]:
#lineáris egyenletrendszer megoldása a numpy csomaggal
import numpy as np
#.linalg-al érhetitek el ezeket a funkciókat.
A=np.array([[1,1,1],[1,2,4],[1,3,6]])
print(A, np.linalg.det(A))
b=np.array([[4],[5],[7]])
#2x2es lineáris egyenletrendszer megoldása
x=np.linalg.solve(A,b)
print(x)
#gyorsan csinálja, gaussozik.

[[1 1 1]
 [1 2 4]
 [1 3 6]] -1.0
[[ 1.]
 [ 4.]
 [-1.]]


Ilyen egyszerű LER-eket megoldanunk! Vizsgára gyakorláshoz jó lesz. Most alkalmazzuk végeselem-módszerrel egy szilárdságtani probléma megoldására. A feladatunk [ezek](https://drive.google.com/file/d/1R-F0pdHobnl-d0ASx1W-4cjRWVpXaMJs/view?usp=sharing) közül a példák közül az első lesz. 

![Bármit](https://i.imgur.com/9rbaqF2.jpg)

## 1. lépés: elemek és csomópontok felvétele

Vegyük fel a csomópontokat és az elemeket, majd tegyük bele az elemekhez tartó csomópontokat egy mátrixba. Még három információval bírunk az elemekről: tudjuk a hosszt, Young-modulust és a keresztmetszetet, tegyük ezeket is bele SI-ben: ebből kiszámolhatjuk az egyes rudak merevségét.

In [1]:
import numpy as np
elements_node = np.array([[0,0,1],[1,1,2],[2,1,2]])
elements_data = np.array([[1,1.e+11, 6.e-05],[2,2.e+11,2.e-05],[3,5.e+10,3.e-05]])

print(elements_node)
print(elements_data)

[[0 0 1]
 [1 1 2]
 [2 1 2]]
[[1.e+00 1.e+11 6.e-05]
 [2.e+00 2.e+11 2.e-05]
 [3.e+00 5.e+10 3.e-05]]


In [2]:
#merevségek számítása
elements_stiffness=(elements_data[:,1]*elements_data[:,2])/elements_data[:,0]
print(elements_stiffness)

[6000000. 2000000.  500000.]


## 2. lépés: elem merevségi mátrixok

Ha megvan az elemek merevsége, kiszámíthatóm az elemek merevségi mátrixát, ami megmutatja, hogy csomópontjaik elmozdulásának hatására mekkora lesz a csomóponti terhelés.

In [3]:
#merevségi mátrixok - az óra alapján:
#elemmerevségi mátrixok "gerince"
element_SMs=np.tile(np.array([[1,-1],[-1,1]]),(3,1,1))
#type átalakítás
element_SMs=element_SMs.astype('float64')
#"broadcasting"-al összepakolhatom, de előtte reshape kell
elements_stiffness=elements_stiffness.reshape(3,1,1)
#broadcasting
element_SMs*=elements_stiffness
print(element_SMs)

[[[ 6000000. -6000000.]
  [-6000000.  6000000.]]

 [[ 2000000. -2000000.]
  [-2000000.  2000000.]]

 [[  500000.  -500000.]
  [ -500000.   500000.]]]


## 3. lépés: globális merevségi mátrix

Az elemek merevségi mátrixaiból összeadjuk a globális merevségi mátrixot úgy, hogy minden elemhez a megfelelő szabadságfokhoz tartozó komponenseket adjuk. 

In [4]:
#mathematicában ez egy sor...
global_stiffness=np.zeros((3,3))
for element in elements_node:
    #típuskonverzió az indexeléshez
    #element=element.astype('uint64')
    #rettenetes randaság
    global_stiffness[element[1],element[1]]+=element_SMs[element[0],0,0]
    global_stiffness[element[1],element[2]]+=element_SMs[element[0],0,1]
    global_stiffness[element[2],element[1]]+=element_SMs[element[0],1,0]
    global_stiffness[element[2],element[2]]+=element_SMs[element[0],1,1]
    
print(global_stiffness) #nézzünk azért rá!

[[ 6000000. -6000000.        0.]
 [-6000000.  8500000. -2500000.]
 [       0. -2500000.  2500000.]]


## 4. lépés: kondenzált mennyiségek meghatározása

A numpy nem egy szimbolikus dolog, szóval most sokminden könnyebb papíron. Keressük meg először azokat a csomópontokat, amelyek elmozdulása ismert! Csak a "2"-es csomópont ilyen, ennek az elmozdulása nulla. Elvileg a többi csomópontban ismernünk kell a terhelést, de nézzük meg, hogy így van-e:
 - Látjuk, hogy a "0"-s csomópontot egy megadott $F_t$ terhelő erő nyomja.
 - Nincs sehova se odaírva, hogy mi hat az "1"-esen, de látjuk, hogy szabadon elmozdul, tehát ott a terhelés zérus.
 
Valóban ismerjük a terhelést ott, ahol az elmozdulást nem tudjuk, és fordítva. A $K\cdot U =F$ merevségi egyenlet kondenzálása annyit tesz, hogy elhagyjuk az egyenletből azokat a szabadságfokokat, amelyekhez ismert elmozdulások tartoznak (most a "2"-es ilyen), és megoldjuk a kapott lineáris egyenletrendszert.

A kondenzált globális merevségi mátrix és terhelésvektor tehát:

In [5]:
condensed_stiffness=global_stiffness[0:2, 0:2]
condensed_load=np.array([[-1.5e+04],[0]])

## 5. lépés: kondenzált merevségi egyenlet megoldása

A bemutatott lineáris egyenleteket megoldó programmal megoldjuk a kapott egyenletet.

In [6]:
condensed_deformation=np.linalg.solve(condensed_stiffness,condensed_load)

In [7]:
print(condensed_deformation)

[[-0.0085]
 [-0.006 ]]


In [8]:
print(np.dot(condensed_stiffness,condensed_deformation)) #ellenőrizhetnek az ijedősebbek

[[-1.50000000e+04]
 [ 7.27595761e-12]]


## 6. lépés: feladat szempontjából érdekes mennyiségek összegyűjtése

A feszültségek (azaz az csomópontokat terhelő erők) érdekesek, valamint a középső rúd elmozdulása. Az utóbbit már megkaptuk, az előbbiekhez kell még sportolni kicsit: az ismert elmozdulásokból elemenként megkapható az elem terhelésvektora, ebből pedig a feszültsége. Ellenőrzésképp összeállítjuk a globális deformációvektort, és megkapjuk a globális terhelést a merevségi egyenletből - elvileg az elemek terheléseinek összege ezt adja.

In [9]:
global_deformation=np.zeros((3,1))
global_deformation[:-1,:]+=condensed_deformation
print(global_deformation)

[[-0.0085]
 [-0.006 ]
 [ 0.    ]]


In [10]:
load_elements=[]
stress_elements=[]
for element in elements_node:
    #típuskonverzió az indexeléshez
    #element=element.astype('uint64')
    #legszebb sor
    element_deformation=np.zeros((2,1))
    element_deformation[0,0]+=global_deformation[element[1]]
    element_deformation[1,0]+=global_deformation[element[2]]
    element_load=np.dot(element_SMs[element[0]],element_deformation)
    load_elements.append(element_load)

for i in range(len(elements_node)):
    element_stress=load_elements[i][0]/elements_data[i,2] ##[][] vagy [,]??
    stress_elements.append(element_stress)
    
print(stress_elements)

[array([-2.5e+08]), array([-6.e+08]), array([-1.e+08])]


In [11]:
global_load=np.dot(global_stiffness, global_deformation)
print(global_load)

##Valóban 0 jön ki. 

[[-1.50000000e+04]
 [ 7.27595761e-12]
 [ 1.50000000e+04]]


## Szorgalmi feladat 10 pontért

Aki az óra anyaga alapján, a feltöltött segédanyagok segítségével elkészíti a másik példát is, és feltölti a [formra](https://forms.gle/MDCssp98TXSsSYkL6), 10 pont üti a markát.